In [5]:
from __future__ import print_function, division
import time
import torch
import pickle
from tqdm import tqdm
import torch.nn as nn
from torch.nn import Parameter
import LocalEnergyVct as le
from torch.utils.data import DataLoader, random_split
from data_classes import RNASeqDataset
# from my_script import get_target, loss_fn, train, test

In [2]:
# CUDA for Pytorch
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
device ='cpu'

True


In [3]:
def get_target(X):
    if len(X['features'].shape) == 2:
        X['features'] = X['features'].unsqueeze(0)
    target = (X['features'][:,0:3,9])
    return target


# Functions without gradients in the loss

def loss_fn(energy,target):
    batch_size = energy.shape[0]
    loss = (energy - target).pow(2).sum() / batch_size
    return loss


def train(dataloader, model, loss_fn, optimizer):
    num_batches = len(dataloader)
    model.train()
    train_loss = 0
    for X in tqdm(dataloader):
        pred = model(X)
        target = get_target(X)
        loss = loss_fn(pred, target)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= num_batches
    print(f'Avg train_loss = {train_loss:>0.4f}, batches = {num_batches}')
    return train_loss


def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X in tqdm(dataloader):
            pred = model(X)
            target = get_target(X)
            loss = loss_fn(pred, target)
            test_loss += loss.item()
    test_loss /= num_batches
    print(f'Avg test_loss = {test_loss:>0.4f}, batches = {num_batches}')
    return test_loss

In [37]:
class LocalEnergyOpt(nn.Module):

    def __init__(self,fixed_pars,opt_pars,device,set_to_one=True):
        super(LocalEnergyOpt, self).__init__()
        if set_to_one:
            fixed_pars['bond_type'][:,0] = 1.
            fixed_pars['angle_type'][:,0] = 1.
            fixed_pars['torsion_type'][:,0] = 1.
        self.device = device
        self.opt_pars = Parameter(torch.tensor(opt_pars, dtype=torch.float, device=self.device, requires_grad=True))
        self.bond_type = Parameter(torch.tensor(fixed_pars['bond_type'], dtype=torch.float, device=self.device, requires_grad=True))
        self.angle_type = Parameter(torch.tensor(fixed_pars['angle_type'], dtype=torch.float, device=self.device, requires_grad=True))
        self.tor_type = Parameter(torch.tensor(fixed_pars['torsion_type'][:,(0,2)], dtype=torch.float, device=self.device, requires_grad=True))
        self.multiplicity = torch.tensor(fixed_pars['torsion_type'][:,1], dtype=torch.int64, device=self.device, requires_grad=False)


    def forward(self,X):

        X_lengths = X['lengths']
        X_features = X['features']
        batch_size = X_lengths.shape[0]

        energy = torch.zeros(X_lengths.shape[0],3,device=self.device)

        print(X_lengths[:,5])
        print(X_features[:,:X_lengths[:,5],5])
        coords = X_features[:,X_lengths[:,5],5].view(batch_size,-1,3)

        print(coords)

        for i in range(X_lengths.shape[0]):
            lengths = X_lengths[i]
            features = X_features[i]
            atoms = features[:lengths[0],0].long()
            # res_labels
            # res_pointer
            # mass
            # charge
            coords = features[:lengths[5],5].view(-1,3)
            bonds = features[:lengths[6],6].long().view(-1,3)
            angles = features[:lengths[7],7].long().view(-1,4)
            tors = features[:lengths[8],8].long().view(-1,5)
            energy[i,0] = le.bonds_energy(coords,bonds,self.bond_type,self.opt_pars)
            energy[i,1] = le.angles_energy(atoms,coords,angles,self.angle_type,self.opt_pars)
            energy[i,2] = le.torsions_energy(atoms,coords,tors,self.tor_type,self.multiplicity,self.opt_pars)

        return energy

In [17]:
# Parameters
params = {'batch_size': 2,
      'shuffle': True,
      'num_workers': 0,
      'pin_memory': False}
epochs = 300

In [23]:
# Datasets and Dataloaders
seq_data = RNASeqDataset(device=device)
print(f'dataset allocated on {device}')

tot_length = len(seq_data)
test_length = int(0.2*tot_length)
train_set, test_set = random_split(seq_data, [tot_length - test_length, test_length], generator=torch.Generator().manual_seed(42))
print(f'Training set: {len(train_set)} elements')
print(f'Test set: {len(test_set)} elements')

dataset allocated on cpu
Training set: 1311 elements
Test set: 327 elements


In [38]:
fixed_pars = pickle.load(open('data/SeqCSV/fixed_pars.p', 'rb'))
opt_pars = pickle.load(open('data/SeqCSV/pars.p', 'rb'))
model = LocalEnergyOpt(fixed_pars,opt_pars,device,set_to_one=False).to(device)
torch.save(model.state_dict(), 'data/Results/initial_values.pth')
# model.load_state_dict(torch.load("data/Results/model_pars.pth"))
# model.load_state_dict(torch.load("data/Results/try_batch16.npy"))

lr = 1e-7
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=500, cooldown=1000, threshold=1e-12, verbose=True)
my_loss = loss_fn  # _with_grad
my_train = train  # _with_grad
my_test = test  # _with_grad

In [29]:
train_loss = []
test_loss = []
train_dataloader = DataLoader(train_set,**params)
test_dataloader = DataLoader(test_set,**params)

In [39]:
for X in tqdm(train_dataloader):
        pred = model(X)
        target = get_target(X)
        loss = loss_fn(pred, target)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  0%|          | 0/656 [00:00<?, ?it/s]

tensor([135, 135])


TypeError: only integer tensors of a single element can be converted to an index